In [13]:
import os
import subprocess
from lxml import etree
import csv

In [14]:
AkWarmExe = r'C:\Program Files (x86)\AHFC\AkWarm\AkWarmCL.exe'
inputDirectory = r"C:\Users\dustin\Desktop\HUGE Random Sample of AkWarm Files\1650 Files (duplicate)"
outputDirectory = r"C:\Users\dustin\Desktop\HUGE Random Sample of AkWarm Files\ReRun Results"
libraryToUse = 'Newest'  # valid choices are Newest, BestMatch, or a filename

In [15]:
class CalculationResults(object):

    def __init__(self, filename):
        self.filename = os.path.basename(filename)
        input_file = filename
        output_file = os.path.join(outputDirectory, os.path.splitext(self.filename)[1] + '.xml')
        self.exitCode = subprocess.call([AkWarmExe, input_file, output_file, libraryToUse])
        if self.exitCode in [0, 1, 2]:
            try:
                self.tree = etree.parse(output_file)
            except:
                self.tree = None
        else:
            self.tree = None

    @staticmethod
    def as_keys():
        return ['FileName',
                'Result',
                'RatingPoints',
                'EnergyCost',
                'ElectricUse',
                'SavingsAnnual',
                'IECCSavingsFract',
                'IECCEnvelopeSavingsFract',
                'HomeAddress',
                'HomeCity', 
                'HouseType',
                'RatingDate',
                'RaterName',
                'HtgSysEffic',
                'UserAFUE',
                'FloorArea',
                'GarageArea',
                'HeatingFuel',
                'DHWFuel']

    def as_dict(self):
        return {'FileName': self.filename,
                'Result': self.exit_code_text(),
                'RatingPoints': self.rating_points(),
                'EnergyCost': self.energy_cost(),
                'ElectricUse': self.electric_use(),
                'SavingsAnnual': self.savings_annual(),
                'IECCSavingsFract': self.IECC_savings(), 
                'IECCEnvelopeSavingsFract': self.envelope_savings(),
                'HomeAddress': self.home_address(),
                'HomeCity': self.home_city(),
                'HouseType': self.house_type(),
                'RatingDate': self.rating_date(),
                'RaterName': self.rater_name(),
                'HtgSysEffic': self.heat_eff(),
                'UserAFUE': self.user_afue(),
                'FloorArea': self.floor_area(),
                'GarageArea': self.garage_area(), 
                'HeatingFuel': self.heat_fuel(),
                'DHWFuel': self.dhw_fuel()
               }

    def exit_code_text(self):
        exit_code_lookup = {0: 'Success',
                            1: 'CalculatedWithValidationErrors',
                            2: 'CalculatedWithValidationWarnings',
                            10: 'CalculationError',
                            20: 'InvalidInputFile',
                            21: 'InvalidLibrary',
                            22: 'ErrorCreatingOutputFile',
                            29: 'OtherProcessingError'}
        if self.exitCode in exit_code_lookup:
            return exit_code_lookup[self.exitCode]
        else:
            return self.exitCode

    def rating_points(self):
        if self.tree:
            try:
                return self.tree.xpath('//RateResults/RatingPoints')[0].text
            except:
                pass
        return None

    def energy_cost(self):
        if self.tree:
            try:
                return self.tree.xpath('//EnrgResults/EnergyCost')[0].text.replace(',', '')
            except:
                pass
        return None

    def electric_use(self):
        if self.tree:
            try:
                return self.tree.xpath('//AnnualQuantityFuel/Electric')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def savings_annual(self):
        if self.tree:
            try:
                return self.tree.xpath('//ImpResults/TotalsCostEffective/SavingsAnnual')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def IECC_savings(self):
        if self.tree:
            try:
                return self.tree.xpath('//RateResults/IECC_TotalSavingsFrac')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def envelope_savings(self):
        if self.tree:
            try:
                return self.tree.xpath('//RateResults/IECC_EnvelopeSavingsFrac')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def home_address(self):
        if self.tree:
            try:
                return self.tree.xpath('//HmInputs/HomeAddr')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def home_city(self):
        if self.tree:
            try:
                return self.tree.xpath('//HmInputs/HomeCity')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def house_type(self):
        if self.tree:
            try:
                return self.tree.xpath('//HmInputs/HouseType')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def rating_date(self):
        if self.tree:
            try:
                return self.tree.xpath('//HmInputs/RatingDateTime')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def rater_name(self):
        if self.tree:
            try:
                return self.tree.xpath('//Rater/FirstLastName')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def heat_eff(self):
        if self.tree:
            try:
                return self.tree.xpath('//EnrgResults/HtgSysEffic')[0].text.replace(',', '')
            except:
                pass
        return None

    def user_afue(self):
        if self.tree:
            try:
                return self.tree.xpath('//SpaceHeaters/Heater/AFUE')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def floor_area(self):
        if self.tree:
            try:
                return self.tree.xpath('//HmInputs/FloorArea/Expression')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def garage_area(self):
        if self.tree:
            try:
                return self.tree.xpath('//HmInputs/GarageArea/Expression')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def heat_fuel(self):
        if self.tree:
            try:
                return self.tree.xpath('//SpaceHeaters/Heater/Fuel')[0].text.replace(',', '')
            except:
                pass
        return None
    
    def dhw_fuel(self):
        if self.tree:
            try:
                return self.tree.xpath('//DHWheaters/DHWheater/Fuel')[0].text.replace(',', '')
            except:
                pass
        return None
    

In [16]:
%%time
with open(os.path.join(outputDirectory, 'ReRunReultsRemaining1650.csv'), 'wb+') as outputCSV:
    outputWriter = csv.DictWriter(outputCSV, CalculationResults.as_keys())
    outputWriter.writeheader()

    for path, dirs, files in os.walk(inputDirectory):
        for fileName in files:
            fileRoot, fileExtension = os.path.splitext(fileName)
            if fileExtension in ['.hm2', '.hom', '.hmc', '.HOM']:
                results = CalculationResults(os.path.join(path, fileName))
                outputWriter.writerow(results.as_dict())

Wall time: 2d 10min 12s
